# Part A

- **Love**: Follow and maintain optimal distance
- **Fear**: Retreat from detected humans
- **Aggressive**: Relentless pursuit
- **Curious**: Cautious investigation with circling

Keyboard controls:
- 'l' = Love behavior
- 'f' = Fear behavior  
- 'a' = Aggressive behavior
- 'c' = Curious behavior
- 's' = Stop

**Step 1: Display Widgets**

Standard widget setup for displaying color and depth images side by side.

In [1]:
import ipywidgets.widgets as widgets
from IPython.display import display
import cv2

# Create two widgets for displaying images
display_color = widgets.Image(format='jpeg', width='45%')
display_depth = widgets.Image(format='jpeg', width='45%')
layout = widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth], layout=layout)
display(sidebyside)

# convert numpy array to jpeg for displaying
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

**Step 2: Load YOLO Model**

load the TensorRT YOLO model for human detection.

In [2]:
from ultralytics import YOLO

model = YOLO("yolo11l_half.engine")

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


**Step 3: Camera System Initialization**

In [3]:
# ZED camera initialization
import traitlets
import numpy as np
import pyzed.sl as sl
import threading
import motors
from traitlets.config.configurable import SingletonConfigurable

# initialize robot motor control
robot = motors.MotorsYukon(mecanum=False)

# Camera class with traitlets
class Camera(SingletonConfigurable):
    color_value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()

        # ZED camera initialization
        self.zed = sl.Camera()
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA  # 672x376
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA
        init_params.coordinate_units = sl.UNIT.MILLIMETER  # Distance in mm

        # Open camera
        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

        self.runtime = sl.RuntimeParameters()
        self.thread_runnning_flag = False

        # Get camera resolution
        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width, self.height, sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C4, sl.MEM.CPU)

    # threaded frame capture
    def _capture_frames(self):
        while(self.thread_runnning_flag == True):
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                # Retrieve images
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
                
                # Convert BGRA to BGR 
                self.color_value_BGRA = self.image.get_data()
                self.color_value = cv2.cvtColor(self.color_value_BGRA, cv2.COLOR_BGRA2BGR)
                # Get depth as numpy array
                self.depth_image = np.asanyarray(self.depth.get_data())
                
    # thread control methods
    def start(self):
        if self.thread_runnning_flag == False:
            self.thread_runnning_flag = True
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False
            self.thread.join()
            robot.stop()

# Initialize and start camera
camera = Camera()
camera.start()

[2025-11-21 11:53:18 UTC][ZED][INFO] Logging level INFO
[2025-11-21 11:53:18 UTC][ZED][INFO] Logging level INFO
[2025-11-21 11:53:18 UTC][ZED][INFO] Logging level INFO
[2025-11-21 11:53:19 UTC][ZED][INFO] [Init]  Depth mode: ULTRA
[2025-11-21 11:53:20 UTC][ZED][INFO] [Init]  Camera successfully opened.
[2025-11-21 11:53:20 UTC][ZED][INFO] [Init]  Camera FW version: 1523
[2025-11-21 11:53:20 UTC][ZED][INFO] [Init]  Video mode: VGA@100
[2025-11-21 11:53:20 UTC][ZED][INFO] [Init]  Serial Number: S/N 35129214
Loading yolo11l_half.engine for TensorRT inference...


**Step 4: Behavior Functions**

In [4]:
# Global variable for current behavior mode
current_behavior = 'stop'

def detect_human(frame):
    """
    Human detection and distance calculation
    
    From Tutorial 5:
    - YOLO detection for humans (class 0)
    - Bounding box extraction: result.boxes.xyxy[i]
    - Center calculation: (bbox[0] + bbox[2]) / 2 for x, (bbox[1] + bbox[3]) / 2 for y
    - Distance extraction: camera.depth_image[center_y, center_x]

    """

    results = model(frame, verbose=False)
    
    # confidence threshold for detections
    conf_threshold = 0.5
    human_detected = False
    min_distance = float('inf')
    closest_bbox = None
    
    # iterate through detections
    for result in results:
        for i in range(len(result.boxes.cls)):
            if result.boxes.cls[i] == 0:
                # Check confidence (Tutorial 5)
                if result.boxes.conf[i] > conf_threshold:
                    human_detected = True
                    # Get bounding box coordinates (Tutorial 5)
                    bbox = result.boxes.xyxy[i]
                    
                    # Calculate center of bounding box (Tutorial 5 method)
                    # bbox format: [x1, y1, x2, y2]
                    center_x = int((bbox[0] + bbox[2]) / 2)
                    center_y = int((bbox[1] + bbox[3]) / 2)
                    
                    # Get depth at human center (Tutorial 5)
                    # Depth image indexing: depth_image[row, col] or depth_image[y, x]
                    # Returns distance in millimeters
                    if (0 <= center_y < camera.height and 0 <= center_x < camera.width):
                        distance = camera.depth_image[center_y, center_x]
                        
                        # NaN means no depth data, negative or zero means invalid
                        if not np.isnan(distance) and distance > 0:
                            # Track closest human
                            if distance < min_distance:
                                min_distance = distance
                                closest_bbox = bbox
    
    return human_detected, min_distance, closest_bbox

def love_behavior(frame, depth_colormap, human_detected, distance, bbox):
    """
    Love behavior - follow human and maintain comfortable distance
    
    Distance thresholds chosen based on testing:
    - < 450mm: Too close, back up
    - 450-600mm: Optimal distance, stop
    - 600-1200mm: Medium range, approach slowly
    - > 1200mm: Far away, approach faster
    
    """
    if human_detected and distance != float('inf'):
        # Draw detection box (Tutorial 5 pattern)
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), 
                     (int(bbox[2]), int(bbox[3])), (255, 105, 180), 2)
        
        # Calculate if human is left/right of center (Tutorial 5 approach)
        bbox_center_x = (bbox[0] + bbox[2]) / 2
        frame_center_x = camera.width / 2
        
        # Distance-based behavior (new thresholds for love behavior)
        if distance < 600:
            robot.backward(0.5)
        elif distance >= 600 and distance <= 1000:
            robot.stop()
        elif distance > 1000 and distance <= 1200:
            # Medium distance - turn to center, then approach
            # 50 pixel threshold to avoid constant adjustment
            if bbox_center_x < frame_center_x - 50:
                robot.left(0.3) 
            elif bbox_center_x > frame_center_x + 50:
                robot.right(0.3) 
            else:
                robot.forward(0.3) 
        else:  # distance > 1200
            # Far away - approach faster with turning
            if bbox_center_x < frame_center_x - 50:
                robot.left(0.4)
            elif bbox_center_x > frame_center_x + 50:
                robot.right(0.4)
            else:
                robot.forward(0.5)
    else:
        # No human detected - stop and wait
        robot.stop()

**Step 5: Keyboard Control and Main Callback**

Based on Answer_keyboard_control.ipynb pattern for keyboard input handling.

In [5]:
text_input = widgets.Text(
    value='',
    placeholder='Type: g=Go, s=Stop',
    description='Behavior:',
    disabled=False
)

# observe text changes
def on_text_change(change):
    """
    Handle keyboard input to switch behaviors
    """
    global current_behavior
    input_value = change['new']
    
    if len(input_value) > 0:
        last_char = input_value[-1].lower()

        # Switch behavior based on input
        if last_char == 'g':
            current_behavior = 'go'
            print('\nSTARTED - Robot started')
        elif last_char == 's':
            current_behavior = 'stop'
            robot.stop()
            print('\nSTOPPED - Robot halted')

text_input.observe(on_text_change, names='value')
display(text_input)

def main_callback(change):
    """
    Main callback function - executes current behavior    
    """
    # Get updated frame
    frame = change['new']
    
    # Create depth colormap for display
    # Converts depth values to color visualization
    depth_colormap = cv2.applyColorMap(
        cv2.convertScaleAbs(camera.depth_image, alpha=0.03), 
        cv2.COLORMAP_JET)
    
    # Detect humans and measure distance (Tutorial 5 method)
    human_detected, distance, bbox = detect_human(frame)
    
    # Execute behavior based on current mode
    if current_behavior == 'go':
        love_behavior(frame, depth_colormap, human_detected, distance, bbox)
    elif current_behavior == 'stop':
        robot.stop()
        cv2.putText(depth_colormap, 'STOPPED', 
                   (250, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (128, 128, 128), 2)
    
    # Display current behavior mode on frame
    cv2.putText(frame, f'Mode: {current_behavior.upper()}', 
               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    # Display images
    scale = 0.3
    resized_color = cv2.resize(frame, None, fx=scale, fy=scale, 
                               interpolation=cv2.INTER_AREA)
    resized_depth = cv2.resize(depth_colormap, None, fx=scale, fy=scale, 
                               interpolation=cv2.INTER_AREA)
    display_color.value = bgr8_to_jpeg(resized_color)
    display_depth.value = bgr8_to_jpeg(resized_depth)

camera.observe(main_callback, names=['color_value'])

print('Press keys to switch modes:')
print('  g = Go (start robot)')
print('  s = Stop (halt robot)')
print('===================================')

Text(value='', description='Behavior:', placeholder='Type: g=Go, s=Stop')

Press keys to switch modes:
  g = Go (start robot)
  s = Stop (halt robot)


**To stop the system:**

In [6]:
# stop camera thread and motors
#camera.stop()
print('System stopped - camera and robot halted')

System stopped - camera and robot halted


## Implementation Notes

### Distance Calculation
Distance is measured by:
1. Getting bounding box from YOLO: `bbox = result.boxes.xyxy[i]` (format: [x1, y1, x2, y2])
2. Calculating center: `center_x = (bbox[0] + bbox[2]) / 2`, `center_y = (bbox[1] + bbox[3]) / 2`
3. Reading depth at center: `distance = camera.depth_image[center_y, center_x]`
4. Depth image returns distance in millimeters


### Behavior Distance Thresholds:

**Love:** 450mm (too close) | 450-600mm (perfect) | 600-1200mm (medium) | >1200mm (far)

**Fear:** 500mm (danger) | 900mm (caution) | 1300mm (alert) | >1300mm (safe)

**Aggressive:** 300mm (ram) | 600mm (engage) | 1000mm (rush) | >1000mm (chase)

**Curious:** 400mm (investigate) | 800mm (approach) | >800mm (interested)